In [1]:
import TextAnalysis as TA

In [3]:
summary = """The paper commemorates the 50th anniversary of Stephen Hawking's groundbreaking discovery of black holes emitting thermal radiation, known as Hawking radiation. The authors use dimensional analysis to derive the Hawking temperature, a fundamental concept in modern astrophysics. They demonstrate that black holes have an absolute temperature, TH, which depends inversely on their mass, MBH. The authors also explore the physical implications of Hawking's discovery, including the evaporation of black holes and their entropy.

The paper begins by introducing the concept of dimensional analysis, a tool used to study physical quantities with different dimensions. The authors then derive the Hawking temperature using dimensional analysis, starting from the assumption that a static black hole is characterized by its mass. They introduce the standard gravitational parameter, Gm, which is essential to the derivation.

The authors then discuss the physical meaning of Hawking's discovery, highlighting the implications of black holes having an absolute temperature. They also explore the concept of entropy, which is proportional to the area of the event horizon. The paper concludes by discussing the challenges of detecting Hawking radiation and the potential applications of analogue gravity, a field that aims to create physical systems that mimic the behavior of gravitational phenomena.

Throughout the paper, the authors use a range of technical terms, including dimensional analysis, Hawking radiation, entropy, and analogue gravity. They also provide a detailed derivation of the Hawking temperature, using mathematical equations and formulas. The paper is written in a clear and concise style, making it accessible to readers with a background in astrophysics and physics.

"""

In [7]:
TA.toptrigrams(summary, "stopwords.txt")

[(('derive', 'hawking', 'temperature'), 2),
 (('black', 'holes', 'absolute'), 2),
 (('holes', 'absolute', 'temperature'), 2),
 (('commemorates', 'anniversary', 'stephen'), 1),
 (('anniversary', 'stephen', 'hawking'), 1),
 (('stephen', 'hawking', 'groundbreaking'), 1),
 (('hawking', 'groundbreaking', 'discovery'), 1),
 (('groundbreaking', 'discovery', 'black'), 1),
 (('discovery', 'black', 'holes'), 1),
 (('black', 'holes', 'emitting'), 1)]

In [ ]:
from typing import Dict, List
import numpy as np
from transformers import AutoTokenizer, AutoModel
from rouge_score import rouge_scorer
from bert_score import BERTScorer
import torch
import spacy

class ScientificMetricsEvaluator:
    def __init__(self):
        """Initialize scientific metrics"""
        # Load general scientific language model
        self.nlp = spacy.load('en_core_web_sm')
        
        # Initialize ROUGE
        self.rouge_scorer = rouge_scorer.RougeScorer(
            ['rouge1', 'rouge2', 'rougeL', 'rougeLsum'],
            use_stemmer=True
        )
        
        # Initialize BERTScore with scientific BERT
        self.bert_scorer = BERTScorer(
            model_type="adsabs/astroBERT",
            num_layers=9,
            batch_size=32,
            nthreads=4,
            all_layers=False,
            idf=False,
            lang='en',
            rescale_with_baseline=False
        )
        

    def _preprocess_text(self, text: str) -> str:
        """Preprocess text while preserving scientific notation and equations"""
        # Basic cleaning
        text = text.strip()
        
        # Handle equations and mathematical expressions
        equation_markers = ['$', '\\[', '\\]', '\\(', '\\)']
        for marker in equation_markers:
            text = text.replace(marker, f" {marker} ")
            
        return ' '.join(text.split())

    def calculate_rouge(self, reference: str, candidate: str) -> Dict[str, float]:
        """Calculate ROUGE scores"""
        # Preprocess text
        reference = self._preprocess_text(reference)
        candidate = self._preprocess_text(candidate)
        
        # Calculate ROUGE scores
        scores = self.rouge_scorer.score(reference, candidate)
        
        # Extract f-measures
        return {
            'rouge1': scores['rouge1'].fmeasure,
            'rouge2': scores['rouge2'].fmeasure,
            'rougeL': scores['rougeL'].fmeasure,
            'rougeLsum': scores['rougeLsum'].fmeasure
        }

    def calculate_bertscore(self, reference: str, candidate: str) -> Dict[str, float]:
        """Calculate BERTScore"""
        # Calculate base BERTScore
        P, R, F1 = self.bert_scorer.score([candidate], [reference])
        
        return {
            'precision': float(P[0]),
            'recall': float(R[0]),
            'f1': float(F1[0])
        }

    def evaluate_summary(self, reference: str, candidate: str) -> Dict[str, float]:
        """Calculate all metrics and combine them"""
        # Calculate individual metrics
        rouge_scores = self.calculate_rouge(reference, candidate)
        bert_scores = self.calculate_bertscore(reference, candidate)
        
        # Combine scores
        final_scores = {
            'rouge1': rouge_scores['rouge1'],
            'rouge2': rouge_scores['rouge2'],
            'rougeL': rouge_scores['rougeL'],
            'rougeLsum': rouge_scores['rougeLsum'],
            'bertscore_precision': bert_scores['precision'],
            'bertscore_recall': bert_scores['recall'],
            'bertscore_f1': bert_scores['f1'],
        }
        
        # Calculate weighted final score
        weights = {
            'rouge2': 0.35,
            'rougeL': 0.25,
            'bertscore_f1': 0.4,
        }
        
        final_scores['weighted_score'] = sum(
            final_scores[metric] * weight
            for metric, weight in weights.items()
        )
        
        return final_scores

def evaluate_scientific_summary(reference_text: str, summary_text: str) -> Dict[str, float]:
    """Convenience function for evaluating scientific summaries"""
    evaluator = ScientificMetricsEvaluator()
    return evaluator.evaluate_summary(reference_text, summary_text)

# Example usage
if __name__ == "__main__":
    reference = """
    The study observed gravitational waves from a binary black hole merger.
    The signal was detected using laser interferometry.
    """
    
    candidate = """
    Gravitational waves were detected from merging black holes using
    laser interferometer measurements.
    """
    
    scores = evaluate_scientific_summary(reference, candidate)
    print(scores)

{'rouge1': 0.5333333333333333, 'rouge2': 0.21428571428571427, 'rougeL': 0.4666666666666666, 'rougeLsum': 0.4666666666666666, 'bertscore_precision': 0.8919382095336914, 'bertscore_recall': 0.8505531549453735, 'bertscore_f1': 0.8707542419433594, 'weighted_score': 0.5399683634440104}


In [12]:
torch.cuda.empty_cache()